# 加载数据集

要运行此笔记本，您需要安装以下库：pandas、openai、transformers、plotly、matplotlib、scikit-learn、torch（transformers 依赖项）、torchvision 和 scipy。

In [ ]:
%pip install pandas
%pip install openai
%pip install transformers
%pip install plotly
%pip install matplotlib
%pip install scikit-learn
%pip install torch
%pip install torchvision
%pip install scipy

本示例中使用的数据集是来自亚马逊的精选食品评论。该数据集总共包含了截至 2012 年 10 月亚马逊用户留下的 568,454 条食品评论。我们将使用该数据集的一个子集，即 1,000 条最新的评论，以便进行说明。这些评论为英文，倾向于正面或负面。每条评论包含产品 ID（ProductId）、用户 ID（UserId）、评分（Score）、评论标题（Summary）以及评论正文（Text）。
我们将把评审总结和评审文本合并为一个综合文本。模型将对这个综合文本进行编码，并输出一个单一的向量嵌入。

In [2]:
import pandas as pd


embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens = 8000  # the maximum for text-embedding-3-small is 8191

#加载和检查数据集
input_datapath = "./archive/2-Reviews.csv"  # 这是我的向量数据目录，同学们下载之后修改这个路径为你自己的
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
# 返回数据框的前两行，用于快速查看数据的初始状态
df.head(2)



,Time,ProductId,UserId,Score,Summary,Text,combined
Id,,,,,,,
1,1303862400,B001E4KFG0,A3SGXH7AUHU8GW,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Title: Good Quality Dog Food; Content: I have ...
2,1346976000,B00813GRG4,A1D87F6ZCVE5NK,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Title: Not as Advertised; Content: Product arr...


In [5]:
#子样本1k最近的评论并删除太长的样本
top_n = 1000
df = df.sort_values("Time").tail(top_n * 2)  # 第一个剪切到第一个2k条目，假设过滤掉不到一半
df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# 省略太长而无法嵌入的评论
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)

1000

# 获取嵌入向量并保存以供将来服用
## Reviews.csv中的全部输入向量化

In [11]:
import os
from openai import OpenAI
from utils.embeddings_utils import get_embedding
client = OpenAI()

# 这可能需要几分钟（我大概用了10分钟）
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, model=embedding_model))
df.to_csv("./archive//fine_food_reviews_with_embeddings_1k.csv")



## 一个更简单的示例

In [ ]:
from openai import OpenAI
client = OpenAI()


a = get_embedding("张三李四王五赵六", model=embedding_model)
print(a)

[-0.003749302588403225, -0.01917334459722042, 0.012175715528428555, 0.03283194825053215, 0.01620756834745407, -0.03725656494498253, -0.02823098748922348, ... 0.003039920935407281]
